# SQLite

## Install Dependency

In [13]:
%pip install sqlite3

%pip install pandas

%pip install rggrader

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
You should consider upgrading via the 'c:\Users\LENOVO\Documents\wiwie\projects\ai-sqlite-and-elastic-search-exercise\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LENOVO\Documents\wiwie\projects\ai-sqlite-and-elastic-search-exercise\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 2.6/2.6 MB 8.6 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-win_amd64.whl (102 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LENOVO\Documents\wiwie\projects\ai-sqlite-and-elastic-search-exercise\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Import Library

In [2]:
import sqlite3

import pandas as pd

## Read Dataset

In [3]:
dataset_csv = pd.read_csv("../datasets/data.csv", encoding='ISO-8859-1')

dataset_csv.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Connect to SQLite

In [4]:
connection = sqlite3.connect('../database/e-commerce.db')
cursor = connection.cursor()

## Create SQLite Table

In [5]:
cursor.execute('''
    CREATE TABLE sales_order (
        InvoiceNo VARCHAR(255) NOT NULL,
        StockCode VARCHAR(255) NOT NULL,
        Description VARCHAR(255),
        Quantity INTEGER NOT NULL,
        InvoiceDate DATETIME NOT NULL,
        UnitPrice FLOAT NOT NULL,
        CustomerID FLOAT,
        Country VARCHAR(255) NOT NULL
    )
''')

connection.commit()

## Insert Data into SQLite Table

In [6]:
data = dataset_csv[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']].values.tolist()

cursor.executemany('''
    INSERT INTO sales_order (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', data)

connection.commit()

## Check sales_order SQLite Table

In [7]:
import sqlite3

conn = sqlite3.connect('../database/e-commerce.db') 
cur = conn.cursor()

cur.execute("PRAGMA table_info(sales_order)") 
sql_columns = [column[1] for column in cur.fetchall()] 
print(f"Columns: {sql_columns}")

cur.execute("SELECT COUNT(*) FROM sales_order") 
sql_num_rows = cur.fetchone()[0] 
print(f"Number of rows: {sql_num_rows}")

conn.close()

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
Number of rows: 541909


## TASK-01: Total number of unique products and unique customers

In [8]:
sql_num_products = pd.read_sql_query('''
    SELECT COUNT(DISTINCT StockCode) AS 'Unique Products' FROM sales_order
''', connection)

sql_num_products = sql_num_products['Unique Products'][0]

display(sql_num_products)

sql_num_customers = pd.read_sql_query('''
    SELECT COUNT(DISTINCT CustomerID) AS 'Unique Customers' FROM sales_order
''', connection)

sql_num_customers = sql_num_customers['Unique Customers'][0]

display(sql_num_customers)

np.int64(4070)

np.int64(4372)

## TASK-02: Total revenue for each product, limited to top 5

In [9]:
sql_top_product_revenues = pd.read_sql_query('''
    SELECT 
        Description,
        SUM(Quantity * UnitPrice) AS Revenue
    FROM sales_order 
    GROUP BY Description 
    ORDER BY Revenue DESC
    LIMIT 5
''', connection)

sql_top_product_revenues = list(sql_top_product_revenues.itertuples(index=False, name=None))

display(sql_top_product_revenues)  

[('DOTCOM POSTAGE', 206245.48000000019),
 ('REGENCY CAKESTAND 3 TIER', 164762.19),
 ('WHITE HANGING HEART T-LIGHT HOLDER', 99668.46999999849),
 ('PARTY BUNTING', 98302.98000000048),
 ('JUMBO BAG RED RETROSPOT', 92356.03000000204)]

## TASK-03: The most profitable countries (by total sales), limited to top 5

In [10]:
sql_top_countries = pd.read_sql_query('''
    SELECT 
        Country,
        SUM(Quantity * UnitPrice) AS Revenue
    FROM sales_order 
    GROUP BY Country 
    ORDER BY Revenue DESC
    LIMIT 5
''', connection)

sql_top_countries = list(sql_top_countries.itertuples(index=False, name=None))
sql_top_country_names = tuple(map(lambda top_country: top_country[0], sql_top_countries))

display(sql_top_countries)  
display(sql_top_country_names)

[('United Kingdom', 8187806.364001113),
 ('Netherlands', 284661.54000000015),
 ('EIRE', 263276.81999999826),
 ('Germany', 221698.20999999862),
 ('France', 197403.90000000002)]

('United Kingdom', 'Netherlands', 'EIRE', 'Germany', 'France')

## TASK-04: Top-selling products (by total sales) for each country, limited to top 5

In [11]:
pd.options.display.float_format = '{:.10f}'.format

sql_top_selling_product = pd.read_sql_query(f'''
    SELECT 
        Country,
        Description,
        Revenue
    FROM (
        SELECT 
            Country,
            Description,
            SUM(Quantity * UnitPrice) AS Revenue,
            ROW_NUMBER() OVER (PARTITION BY Country ORDER BY SUM(Quantity * UnitPrice) DESC) AS row_num
        FROM sales_order
        WHERE country IN {sql_top_country_names}
        GROUP BY Country, Description
    ) AS ranked_sales
    WHERE row_num <= 5
    ORDER BY 
        CASE Country
            WHEN 'United Kingdom' THEN 1
            WHEN 'Netherlands' THEN 2
            WHEN 'EIRE' THEN 3
            WHEN 'Germany' THEN 4
            WHEN 'France' THEN 5
            ELSE 6
        END,
        Revenue DESC;
''', connection)

sql_top_selling_products_by_country_by_sales = {}

for row in sql_top_selling_product.itertuples(index=False):
    country = row.Country
    description = row.Description
    revenue = row.Revenue
    
    if country not in sql_top_selling_products_by_country_by_sales:
        sql_top_selling_products_by_country_by_sales[country] = []
    
    sql_top_selling_products_by_country_by_sales[country].append((description, revenue))

display(sql_top_selling_products_by_country_by_sales)

{'United Kingdom': [('DOTCOM POSTAGE', 206245.48000000019),
  ('REGENCY CAKESTAND 3 TIER', 134405.93999999992),
  ('WHITE HANGING HEART T-LIGHT HOLDER', 93953.0699999987),
  ('PARTY BUNTING', 92501.73000000033),
  ('JUMBO BAG RED RETROSPOT', 84516.44000000166)],
 'Netherlands': [('RABBIT NIGHT LIGHT', 9568.48),
  ('ROUND SNACK BOXES SET OF4 WOODLAND ', 7991.400000000001),
  ('SPACEBOY LUNCH BOX ', 7485.5999999999985),
  ('DOLLY GIRL LUNCH BOX', 6828.599999999999),
  ('ROUND SNACK BOXES SET OF 4 FRUITS ', 4039.2000000000007)],
 'EIRE': [('REGENCY CAKESTAND 3 TIER', 7442.849999999997),
  ('CARRIAGE', 5175.0),
  ('JAM MAKING SET WITH JARS', 3089.0),
  ('3 TIER CAKE TIN RED AND CREAM', 3041.5499999999997),
  ('WHITE HANGING HEART T-LIGHT HOLDER', 2857.8)],
 'Germany': [('POSTAGE', 20821.0),
  ('REGENCY CAKESTAND 3 TIER', 8257.349999999997),
  ('ROUND SNACK BOXES SET OF4 WOODLAND ', 3554.6999999999985),
  ('ROUND SNACK BOXES SET OF 4 FRUITS ', 1949.9500000000016),
  ('SPACEBOY LUNCH BOX ', 

## TASK-05: Top-selling products (by quantity) for each country, limited to top 5

In [12]:
sql_top_selling_product = pd.read_sql_query(f'''
    SELECT 
        Country,
        Description,
        Quantity
    FROM (
        SELECT 
            Country,
            Description,
            SUM(Quantity) AS Quantity,
            ROW_NUMBER() OVER (PARTITION BY Country ORDER BY SUM(Quantity) DESC) AS row_num
        FROM sales_order
        WHERE country IN {sql_top_country_names}
        GROUP BY Country, Description
    ) AS ranked_sales
    WHERE row_num <= 5
    ORDER BY 
        CASE Country
            WHEN 'United Kingdom' THEN 1
            WHEN 'Netherlands' THEN 2
            WHEN 'EIRE' THEN 3
            WHEN 'Germany' THEN 4
            WHEN 'France' THEN 5
            ELSE 6
        END,
        Quantity DESC;
''', connection)

sql_top_selling_products_by_country_by_quantity = {}

for row in sql_top_selling_product.itertuples(index=False):
    country = row.Country
    description = row.Description
    quantity = row.Quantity
    
    if country not in sql_top_selling_products_by_country_by_quantity:
        sql_top_selling_products_by_country_by_quantity[country] = []
    
    sql_top_selling_products_by_country_by_quantity[country].append((country + ' ' + description, quantity))

display(sql_top_selling_products_by_country_by_quantity)

{'United Kingdom': [('United Kingdom WORLD WAR 2 GLIDERS ASSTD DESIGNS',
   48326),
  ('United Kingdom JUMBO BAG RED RETROSPOT', 43167),
  ('United Kingdom POPCORN HOLDER', 34365),
  ('United Kingdom ASSORTED COLOUR BIRD ORNAMENT', 33679),
  ('United Kingdom WHITE HANGING HEART T-LIGHT HOLDER', 33193)],
 'Netherlands': [('Netherlands RABBIT NIGHT LIGHT', 4801),
  ('Netherlands SPACEBOY LUNCH BOX ', 4528),
  ('Netherlands DOLLY GIRL LUNCH BOX', 4132),
  ('Netherlands PACK OF 72 RETROSPOT CAKE CASES', 4128),
  ('Netherlands ROUND SNACK BOXES SET OF4 WOODLAND ', 3132)],
 'EIRE': [('EIRE PACK OF 72 RETROSPOT CAKE CASES', 1728),
  ('EIRE 60 TEATIME FAIRY CAKE CASES', 1536),
  ('EIRE VINTAGE SNAP CARDS', 1492),
  ('EIRE ASSORTED INCENSE PACK', 1440),
  ('EIRE PACK OF 60 PINK PAISLEY CAKE CASES', 1296)],
 'Germany': [('Germany ROUND SNACK BOXES SET OF4 WOODLAND ', 1218),
  ('Germany ASSORTED COLOURS SILK FAN', 1164),
  ('Germany POSTAGE', 1104),
  ('Germany WOODLAND CHARLOTTE BAG', 1019),
  (

## Submission

In [ ]:
student_id = "REAINTCZ" 
name = "Wiwie Sanjaya"
drive_link = "https://github.com/wiwiewei18/ai-sqlite-and-elastic-search-exercise"  

assignment_id = "00_database_project"

from rggrader import submit, submit_image

# question_id = "01_sql_columns"
# submit(student_id, name, assignment_id, str(sql_columns), question_id, drive_link)
# question_id = "02_sql_num_rows"
# submit(student_id, name, assignment_id, str(sql_num_rows), question_id, drive_link)
# question_id = "03_sql_num_products"
# submit(student_id, name, assignment_id, str(sql_num_products), question_id, drive_link)
# question_id = "04_sql_num_customers"
# submit(student_id, name, assignment_id, str(sql_num_customers), question_id, drive_link)
# question_id = "05_sql_top_product_revenues"
# submit(student_id, name, assignment_id, str(sql_top_product_revenues), question_id, drive_link)
# question_id = "06_sql_top_countries"
# submit(student_id, name, assignment_id, str(sql_top_countries), question_id, drive_link)
# question_id = "07_sql_top_selling_products_by_country_by_sales"
# submit(student_id, name, assignment_id, str(sql_top_selling_products_by_country_by_sales), question_id, drive_link)
# question_id = "08_sql_top_selling_products_by_country_by_quantity"
# submit(student_id, name, assignment_id, str(sql_top_selling_products_by_country_by_quantity), question_id, drive_link)